Valuing Actions by Estimating Probabilities

In [1]:
import pandas as pd
import numpy as np
import pandera as pa
from pandera.typing import Series, DataFrame
from typing import Any, List

import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")
from config import chain_file_path

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

Load Chain Data

In [2]:
chains = pd.read_csv(chain_file_path)
chains.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
1169201,252,ballUp,turnover,1984.0,4,1937,1942.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Kick,-65.0,14.0,clanger,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169202,252,ballUp,turnover,1985.0,4,1937,1945.0,St Kilda,St Kilda,Dan Butler,Dan_Butler,Out On Full After Kick,-69.0,34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169203,253,possGain,behind,1986.0,4,1949,1949.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,OOF Kick In,67.0,-34.0,NaN,NaN,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169204,253,possGain,behind,1987.0,4,1949,1976.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Kick,67.0,-34.0,ineffective,True,NaN,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0
1169205,253,possGain,behind,1988.0,4,1949,1978.0,North Melbourne,North Melbourne,Luke Davies-Uniacke,Luke_Davies-Uniacke,Behind,67.0,-34.0,NaN,NaN,missLeft,129,160,St Kilda,North Melbourne,left,202319_StKilda_NorthMelbourne,202319,NaN,2023.0


In [3]:
def play_left_to_right(chains):
    
    chains['new_x'] = chains['x'].copy()
    chains['new_x'] = np.where((chains['Team'] == chains['Home_Team']) & (chains['Home_Team_Direction_Q1'] == "left"), -1*chains['x'], chains['new_x'])
    chains['new_x'] = np.where((chains['Team'] == chains['Away_Team']) & (chains['Home_Team_Direction_Q1'] == "right"), -1*chains['x'], chains['new_x'])

    chains['new_y'] = chains['y'].copy()
    chains['new_y'] = np.where((chains['Team'] == chains['Home_Team']) & (chains['Home_Team_Direction_Q1'] == "left"), -1*chains['y'], chains['new_y'])
    chains['new_y'] = np.where((chains['Team'] == chains['Away_Team']) & (chains['Home_Team_Direction_Q1'] == "right"), -1*chains['y'], chains['new_y'])
        
    return chains

In [4]:
chains = play_left_to_right(chains)

In [5]:
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,new_x,new_y
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0.0,0.0
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,8.0,-5.0
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,9.0,-6.0
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,-11.0,7.0
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,-12.0,5.0


In [6]:
max_quarter_durations = chains.groupby(['Match_ID', "Quarter"]).max()['Quarter_Duration'].reset_index()
max_quarter_durations = max_quarter_durations.rename(columns = {'Quarter_Duration':'Quarter_Duration_Max'})
max_quarter_durations = max_quarter_durations.pivot('Match_ID', 'Quarter', 'Quarter_Duration_Max')
chains = chains.merge(max_quarter_durations, how='left', on = ['Match_ID'])
chains['Duration'] = np.where(chains['Quarter'] == 1, chains['Quarter_Duration'],
                              np.where(chains['Quarter'] == 2, chains[1] + chains['Quarter_Duration'],
                                       np.where(chains['Quarter'] == 3, chains[1] + chains[2] + chains['Quarter_Duration'],
                                                np.where(chains['Quarter'] == 4, chains[1] + chains[2] + chains[3] + chains['Quarter_Duration'],
                                                        0))))


In [7]:
pitch_length: float = 165.0
pitch_width: float = 135.0

goal_width: float = 6.4
behind_width: float = 6.4

left = -pitch_length/2
right = pitch_length/2
top = pitch_width/2
bottom = -pitch_width/2

In [8]:
class AFLSchema(pa.SchemaModel):
    """ Definition of a FADL dataframe. """
    
    match_id: Series[str]
    quarter: Series[int]
    time_seconds: Series[float]
    team: Series[str]
    player: Series[str]
    start_x: Series[float] = pa.Field(ge=left, le=right)
    start_y: Series[float] = pa.Field(ge=bottom, le=top)
    end_x: Series[float] = pa.Field(ge=left, le=right)
    end_y: Series[float] = pa.Field(ge=bottom, le=top)
    action_type: Series[str]
    outcome_type: Series[bool]   

In [9]:
AFLActions = DataFrame[AFLSchema]
GameStates = List[AFLActions]
Features = DataFrame[Any]

In [10]:
action_types = [
    'Kick', # Regain possession
    'Handball', # Regain possesion
    'Carry', # always
    'Uncontested Mark', # Complete mark
    'Contested Mark', # Complete mark
    'Loose Ball Get', # Always success
    'Hard Ball Get', # Always success
    'Spoil', # Always success
    'Gather', # Always success
    'Free For', # Reaches teammate
    'Knock On', # Reaches teammate
    'Shot', # Goal
    'Tackle', # Regain possession
    'Error', # Always fail
]

In [11]:
def get_action_types(chains):
    
    schema_chains = chains.copy()

    schema_chains[~((schema_chains['Description'] == "Contested Mark") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Uncontested Mark") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Contested Knock On") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Gather from Opposition") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Loose Ball Get") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Hard Ball Get") & (schema_chains['Team'] != schema_chains['Team_Chain']))]
    schema_chains[~((schema_chains['Description'] == "Pack Mark (O)") & (schema_chains['Team'] != schema_chains['Team_Chain']))]

    schema_chains['action_type'] = schema_chains['Description'].copy()
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Handball Received", "Carry", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Kickin play on", "Carry", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Bounce", "Carry", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Gather From Hitout", "Gather", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Gather from Opposition", "Gather", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Mark On Lead", "Uncontested Mark", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Hard Ball Get Crumb", "Hard Ball Get", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Loose Ball Get Crumb", "Loose Ball Get", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Ruck Hard Ball Get", "Hard Ball Get", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Free For: In Possession", "Free For", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Free For: Off The Ball", "Free For", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Free For: Before the Bounce", "Free For", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Kickin short", "Kick", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Pack Mark (P)", "Contested Mark", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Pack Mark (O)", "Contested Mark", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "No Pressure Error", "Error", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Debit", "Error", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Contested Knock On", "Knock On", schema_chains['action_type'])
    schema_chains['action_type'] = np.where(schema_chains['Description'] == "Ground Kick", "Kick", schema_chains['action_type'])

    schema_chains['action_type'] = np.where((schema_chains['Description'] == "Kick") & (schema_chains['Shot_At_Goal'] == True), "Shot", schema_chains['action_type'])

    return schema_chains['action_type']

In [12]:
def get_outcome_types(chains):
    
    schema_chains = chains.copy()
    schema_chains['NextTeam'] = schema_chains.groupby('Match_ID')['Team'].shift(-1).fillna(0)
    schema_chains['outcome_type'] = "effective"
    
    schema_chains['outcome_type'] = np.where(schema_chains['action_type'].isin(["Kick", "Handball", "Shot"]), schema_chains['Disposal'], schema_chains['outcome_type'])
    schema_chains['outcome_type'] = np.where((schema_chains['action_type'].isin(["Free For", "Knock On"])) & (schema_chains['Team'] != schema_chains['NextTeam']), "ineffective", schema_chains['outcome_type'])
    schema_chains['outcome_type'] = np.where(schema_chains['action_type'] == "Error", "ineffective", schema_chains['outcome_type'])
            
    return schema_chains['outcome_type']   

In [13]:
def convert_chains_to_schema(chains):
    
    schema_chains = chains.copy()

    schema_chains['match_id'] = schema_chains['Match_ID']
    schema_chains['quarter'] = schema_chains['Quarter']
    schema_chains['quarter_seconds'] = schema_chains['Quarter_Duration']
    schema_chains['overall_seconds'] = schema_chains['Duration']
    schema_chains['team'] = schema_chains['Team']
    schema_chains['player'] = schema_chains['Player']

    schema_chains['start_x'] = schema_chains['new_x']
    schema_chains['start_y'] = schema_chains['new_y']
    schema_chains['end_x'] = schema_chains.groupby('Match_ID')['new_x'].shift(-1).fillna(0)
    schema_chains['end_y'] = schema_chains.groupby('Match_ID')['new_y'].shift(-1).fillna(0)

    schema_chains['action_type'] = get_action_types(schema_chains)
    schema_chains['outcome_type'] = get_outcome_types(schema_chains)

    schema_chains = schema_chains.dropna(subset=['Player'])
    schema_chains = schema_chains[schema_chains['action_type'].isin(action_types)]

    schema_chains = schema_chains[['match_id', 'quarter', 'quarter_seconds', 'overall_seconds', 'team', 'player', 'start_x', 'start_y', 'end_x', 'end_y', 'action_type', 'outcome_type']]
    
    return schema_chains


In [14]:
schema_chains = convert_chains_to_schema(chains)

In [15]:
schema_chains.tail()

,match_id,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type
1169198,202319_StKilda_NorthMelbourne,4,1937.0,7226.0,North Melbourne,Todd Goldstein,-66.0,16.0,-66.0,16.0,Hard Ball Get,effective
1169199,202319_StKilda_NorthMelbourne,4,1937.0,7226.0,North Melbourne,Todd Goldstein,-66.0,16.0,65.0,-14.0,Handball,ineffective
1169200,202319_StKilda_NorthMelbourne,4,1941.0,7230.0,St Kilda,Dan Butler,65.0,-14.0,65.0,-14.0,Loose Ball Get,effective
1169201,202319_StKilda_NorthMelbourne,4,1942.0,7231.0,St Kilda,Dan Butler,65.0,-14.0,69.0,-34.0,Kick,clanger
1169204,202319_StKilda_NorthMelbourne,4,1976.0,7265.0,North Melbourne,Luke Davies-Uniacke,67.0,-34.0,67.0,-34.0,Shot,ineffective


Convert actions to Game States

In [16]:
def gamestates(actions: AFLActions, num_prev_actions: int = 3) -> GameStates:
    """ Convert a dataframe of actions to gamestates.
    
    Each gamestate is represented as the num_prev_actions previous actions.

    Parameters
    ----------
    actions : AFLActions
        A DataFrame with the actions of a game.
    num_prev_actions : int, default = 3
        The number of previous actions included in the gamestate.

    Returns
    -------
    GameStates
        The num_prev_actions previous actions for each action.
    """
    
    if num_prev_actions < 1:
        raise ValueError('The gamestate should include at least one preceding action.')
    
    states = [actions]
    for i in range(1, num_prev_actions):
        prev_actions = actions.copy().shift(i, fill_value=0)
        prev_actions.iloc[:i] = pd.concat([actions[:1]] * i, ignore_index=True)
        states.append(prev_actions)
        
    return states

Create Game State Features

In [17]:
outcome_types = [
    'effective',
    'ineffective',
    'clanger'
]

In [18]:
def action_type(actions):
    
    return actions[['action_type']]

def action_type_onehot(actions):
    
    X = {}
    for action_type in action_types:
        col = 'type_' + action_type
        X[col] = actions['action_type'] == action_type
    return pd.DataFrame(X, index=actions.index)

def outcome(actions):
    
    return actions[['outcome_type']]

def outcome_onehot(actions):
    
    X = {}
    for outcome_type in outcome_types:
        col = 'outcome_' + outcome_type
        X[col] = actions['outcome_type'] == outcome_type
    return pd.DataFrame(X, index=actions.index)

def action_outcome_onehot(actions):
    
    action_type = action_type_onehot(actions)
    outcome_type = outcome_onehot(actions)
    X = {}
    for type_col in list(action_type):
        for outcome_col in list(outcome_type):
            X[type_col + '_' + outcome_col] = action_type[type_col] & outcome_type[outcome_col]
    return pd.DataFrame(X, index=actions.index)

def time(actions):
        
    return actions[['quarter', 'quarter_seconds', 'overall_seconds']].copy()

def start_location(actions):
    
    return actions[['start_x', 'start_y']]

def end_location(actions):
    
    return actions[['end_x', 'end_y']]

def movement(actions):
    
    move = pd.DataFrame(index=actions.index)
    move['dx'] = actions['end_x'] - actions['start_x']
    move['dy'] = actions['end_y'] - actions['start_y']
    move['movement'] = np.sqrt(move['dx']**2 + move['dy']**2)
    return move

def team(gamestates):
    """ Check whether the possession changed during the game state. 
    
    For each action, True if the team that performed the action is the same team that performed the last action.
    Otherwise False
    """
    
    a0 = gamestates[0]
    team_df = pd.DataFrame(index=a0.index)
    for i, a in enumerate(gamestates[1:]):
        team_df['team_'+(str(i+1))] = a['team'] == a0['team']
    return team_df

def time_delta(gamestates):
    """ Get the number of seconds between the last and previous actions. 
    
    """
  
    a0 = gamestates[0]
    dt = pd.DataFrame(index=a0.index)
    for i, a in enumerate(gamestates[1:]):
        dt['time_delta'+(str(i+1))] = a['overall_seconds'] - a0['overall_seconds']
    return dt
    
def space_delta(gamestates):
    """ Get the distance covered between the last and previous actions. 
    
    """
  
    a0 = gamestates[0]
    space_delta = pd.DataFrame(index=a0.index)
    for i, a in enumerate(gamestates[1:]):
        dx = a['end_x'] - a0['start_x']
        space_delta['dx_a0' + (str(i+1))] = dx
        dy = a['end_y'] - a0['start_y']
        space_delta['dy_a0' + (str(i+1))] = dy
        space_delta['move_a0' + str(i+1)] = np.sqrt(dx**2 + dy**2)
        
    return space_delta    

def goal_score(gamestates):
    """ Get the number of goals scored by each team after the action. """
    
    actions = gamestates[0]
    teamA = actions['team'].values[0]
    goals = actions['action_type'].str.contains('Shot') & (actions['outcome_type'] == "effective")
    
    teamisA = actions['team'] == teamA
    teamisB = ~teamisA
    goals_teamA = (goals & teamisA)
    goals_teamB = (goals & teamisB)
    
    goal_score_teamA = goals_teamA.cumsum() - goals_teamA
    goal_score_teamB = goals_teamB.cumsum() - goals_teamB
    
    score_df = pd.DataFrame(index=actions.index)
    score_df['goalscore_team'] = (goal_score_teamA * goals_teamA) + (goal_score_teamB * goals_teamB)
    score_df['goalscore_opponent'] = (goal_score_teamA * goals_teamB) + (goal_score_teamA * goals_teamB)
    score_df['goalscore_diff'] = score_df['goalscore_team'] - score_df['goalscore_opponent']

    return score_df


In [21]:
def create_match_gamestate_features(actions, match_id, num_prev_actions=3):
    
    match_actions = actions[actions['match_id'] == match_id]

    states = gamestates(match_actions, num_prev_actions)
    
    states_features = []
    for actions in range(len(states)):
        state = pd.concat([
            action_type(states[actions]),
            action_type_onehot(states[actions]),
            outcome(states[actions]),
            outcome_onehot(states[actions]),
            action_outcome_onehot(states[actions]),
            time(states[actions]),
            start_location(states[actions]),
            end_location(states[actions]),
            movement(states[actions])
        ], axis=1)
        state.columns = [x+'_a'+str(actions) for x in list(state.columns)]
        states_features.append(state)
        
    features = pd.concat([
        team(states),
        time_delta(states),
        space_delta(states),
        goal_score(states)
        ], axis=1)
    
    states_features.append(features)
    
    gamestate_features = pd.concat(states_features, axis=1) 
    
    return gamestate_features

In [22]:
match_id_list = list(schema_chains['match_id'].unique())
match_gamestate_feature_list = []
for match in match_id_list:
    match_gamestate_features = create_match_gamestate_features(schema_chains, match_id=match, num_prev_actions=3)
    match_gamestate_feature_list.append(match_gamestate_features)
    
gamestate_features = pd.concat(match_gamestate_feature_list, axis=0)

In [23]:
gamestate_features.head()

,action_type_a0,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Uncontested Mark_a0,type_Contested Mark_a0,type_Loose Ball Get_a0,type_Hard Ball Get_a0,type_Spoil_a0,type_Gather_a0,type_Free For_a0,type_Knock On_a0,type_Shot_a0,type_Tackle_a0,type_Error_a0,outcome_type_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Uncontested Mark_outcome_effective_a0,type_Uncontested Mark_outcome_ineffective_a0,type_Uncontested Mark_outcome_clanger_a0,type_Contested Mark_outcome_effective_a0,type_Contested Mark_outcome_ineffective_a0,type_Contested Mark_outcome_clanger_a0,type_Loose Ball Get_outcome_effective_a0,type_Loose Ball Get_outcome_ineffective_a0,type_Loose Ball Get_outcome_clanger_a0,type_Hard Ball Get_outcome_effective_a0,type_Hard Ball Get_outcome_ineffective_a0,type_Hard Ball Get_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,type_Gather_outcome_effective_a0,type_Gather_outcome_ineffective_a0,type_Gather_outcome_clanger_a0,type_Free For_outcome_effective_a0,type_Free For_outcome_ineffective_a0,type_Free For_outcome_clanger_a0,type_Knock On_outcome_effective_a0,type_Knock On_outcome_ineffective_a0,type_Knock On_outcome_clanger_a0,type_Shot_outcome_effective_a0,type_Shot_outcome_ineffective_a0,type_Shot_outcome_clanger_a0,type_Tackle_outcome_effective_a0,type_Tackle_outcome_ineffective_a0,type_Tackle_outcome_clanger_a0,type_Error_outcome_effective_a0,type_Error_outcome_ineffective_a0,type_Error_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,action_type_a1,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Uncontested Mark_a1,type_Contested Mark_a1,type_Loose Ball Get_a1,type_Hard Ball Get_a1,type_Spoil_a1,type_Gather_a1,type_Free For_a1,type_Knock On_a1,type_Shot_a1,type_Tackle_a1,type_Error_a1,outcome_type_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Uncontested Mark_outcome_effective_a1,type_Uncontested Mark_outcome_ineffective_a1,type_Uncontested Mark_outcome_clanger_a1,type_Contested Mark_outcome_effective_a1,type_Contested Mark_outcome_ineffective_a1,type_Contested Mark_outcome_clanger_a1,type_Loose Ball Get_outcome_effective_a1,type_Loose Ball Get_outcome_ineffective_a1,type_Loose Ball Get_outcome_clanger_a1,type_Hard Ball Get_outcome_effective_a1,type_Hard Ball Get_outcome_ineffective_a1,type_Hard Ball Get_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,type_Gather_outcome_effective_a1,type_Gather_outcome_ineffective_a1,type_Gather_outcome_clanger_a1,type_Free For_outcome_effective_a1,type_Free For_outcome_ineffective_a1,type_Free For_outcome_clanger_a1,type_Knock On_outcome_effective_a1,type_Knock On_outcome_ineffective_a1,type_Knock On_outcome_clanger_a1,type_Shot_outcome_effective_a1,type_Shot_outcome_ineffective_a1,type_Shot_outcome_clanger_a1,type_Tackle_outcome_effective_a1,type_Tackle_outcome_ineffective_a1,type_Tackle_outcome_clanger_a1,type_Error_outcome_effective_a1,type_Error_outcome_ineffective_a1,type_Error_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,action_type_a2,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Uncontested Mark_a2,type_Contested Mark_a2,type_Loose Ball Get_a2,type_Ha

Create Response (Scoring and Conceding)

In [24]:
def scores(actions, num_actions: int = 10) -> pd.DataFrame:
    """Determines whether the team possessing the ball scored a goal within the next x actions.
    

    Parameters
    ----------
    actions : pd.DataFrame
        The actions of a game
    num_actions : int, default=10
        Number of actions after the current action to consider.

    Returns
    -------
    pd.DataFrame
        A dataframe with a column 'scores' and a row for each action set to True if a goal was
        scored by the team possessing the ball within the next x actions.
        Otherwise False.
    """
    
    goals = (actions['action_type'] == "Shot") & (actions['outcome_type'] == "effective")
    y = pd.concat([goals, actions['team']], axis=1)
    y.columns = ['goal', 'team']
    
    for i in range(num_actions):
        for c in ['team', 'goal']:
            shifted = y[c].shift(-i).fillna(0)
            # shifted[-i:] = y[c][len(y)-1]
            y['%s+%d' % (c, i)] = shifted
    res = y['goal']
    for i in range(1, num_actions):
        gi = y['goal+%d' % i] & (y['team+%d' % i] == y['team'])
        res = res | gi
    
    return pd.DataFrame(res, columns=['scores'])

def concedes(actions, num_actions: int = 10) -> pd.DataFrame:
    """Determines whether the team possessing the ball conceded a goal within the next x actions.
    

    Parameters
    ----------
    actions : pd.DataFrame
        The actions of a game
    num_actions : int, default=10
        Number of actions after the current action to consider.

    Returns
    -------
    pd.DataFrame
        A dataframe with a column 'concedes' and a row for each action set to True if a goal was
        conceded by the team possessing the ball within the next x actions.
        Otherwise False.
    """
    
    goals = (actions['action_type'] == "Shot") & (actions['outcome_type'] == "effective")
    y = pd.concat([goals, actions['team']], axis=1)
    y.columns = ['goal', 'team']
    
    for i in range(num_actions):
        for c in ['team', 'goal']:
            shifted = y[c].shift(-i).fillna(0)
            y['%s+%d' % (c, i)] = shifted
    res = y['goal']
    res = False
    for i in range(1, num_actions):
        gi = y['goal+%d' % i] & (y['team+%d' % i] != y['team'])
        res = res | gi
    
    return pd.DataFrame(res, columns=['concedes'])


In [25]:
def create_match_gamestate_labels(actions, match_id, num_future_actions=10):
    
    match_actions = actions[actions['match_id'] == match_id]

    gamestate_labels = pd.concat([
        scores(match_actions),
        concedes(match_actions)
    ], axis=1)
    
    return gamestate_labels

In [26]:
create_match_gamestate_labels(schema_chains, match_id='202319_StKilda_NorthMelbourne')

,scores,concedes
1167219,False,False
1167220,False,False
1167221,False,False
1167222,False,False
1167223,False,False
...,...,...
1169198,False,False
1169199,False,False
1169200,False,False
1169201,False,False


In [27]:
match_id_list = list(schema_chains['match_id'].unique())
match_gamestate_label_list = []
for match in match_id_list:
    match_gamestate_labels = create_match_gamestate_labels(schema_chains, match_id=match, num_future_actions=10)
    match_gamestate_label_list.append(match_gamestate_labels)
    
gamestate_labels = pd.concat(match_gamestate_label_list, axis=0)

In [28]:
gamestate_labels

,scores,concedes
1,True,False
2,True,False
3,False,True
4,False,True
5,False,True
...,...,...
1169198,False,False
1169199,False,False
1169200,False,False
1169201,False,False


In [30]:
vaep_modelling_data = pd.concat([schema_chains, gamestate_features, gamestate_labels], axis=1)

In [29]:
def get_stratified_train_test_val_columns(data, response):
    from sklearn.model_selection import train_test_split

    X, y = data.drop(columns=[response]), data[response]
    X_modelling, X_test, y_modelling, y_test = train_test_split(X, y, test_size = 0.2, random_state=2407, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_modelling, y_modelling, test_size = 0.2, random_state=2407, stratify=y_modelling)
    X_train[response+'TrainingSet'] = True
    X_test[response+'TestSet'] = True
    X_val[response+'ValidationSet'] = True
    
    if [response+'TrainingSet', response+'TestSet', response+'ValidationSet'] not in list(data):
        data = pd.merge(data, X_train[response+'TrainingSet'], how="left", left_index=True, right_index=True) 
        data = pd.merge(data, X_test[response+'TestSet'], how="left", left_index=True, right_index=True) 
        data = pd.merge(data, X_val[response+'ValidationSet'], how="left", left_index=True, right_index=True)
        data[[response+'TrainingSet', response+'TestSet', response+'ValidationSet']] = data[[response+'TrainingSet', response+'TestSet', response+'ValidationSet']].fillna(False) 
        
    return data

In [32]:
vaep_modelling_data = get_stratified_train_test_val_columns(vaep_modelling_data, response="scores")
vaep_modelling_data = get_stratified_train_test_val_columns(vaep_modelling_data, response="concedes")

In [33]:
vaep_modelling_data[vaep_modelling_data['scoresTrainingSet']]['scores'].mean(), vaep_modelling_data[vaep_modelling_data['scoresTestSet']]['scores'].mean(), vaep_modelling_data[vaep_modelling_data['scoresValidationSet']]['scores'].mean()

(0.11534220065375746, 0.11534038759771703, 0.11534201052340072)

In [34]:
vaep_modelling_data[vaep_modelling_data['concedesTrainingSet']]['concedes'].mean(), vaep_modelling_data[vaep_modelling_data['concedesTestSet']]['concedes'].mean(), vaep_modelling_data[vaep_modelling_data['concedesValidationSet']]['concedes'].mean()

(0.022888104614380355, 0.02288767684013968, 0.022886418483206078)

In [35]:
vaep_modelling_data.to_csv("../data/vael_modelling_data.csv", index=False)